# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Stored Procedures](#stored-procedures)**
3. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/common/schemas.ipynb"
    ]
}
```

# **TABLES**

---

### STAGINGS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE TABLE(S).....
DROP TABLE IF EXISTS [VWSSTAGE].[RISK_LEVEL_THRESHOLDS]

CREATE TABLE [VWSSTAGE].[RISK_LEVEL_THRESHOLDS] (
    [ID] [INT] PRIMARY KEY IDENTITY(1,1),
    [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[RISK_LEVEL] [VARCHAR](100) NULL,
    [DESCRIPTION] [VARCHAR](100) NULL,
    [7_DAYS_AVERAGE_IC_RECORDINGS] VARCHAR(100) NULL,
    [7_DAYS_AVERAGE_HOSPITALIZATIONS] VARCHAR(100) NULL,
    [VALID_FROM] VARCHAR(100) NULL
);

-- 3) CREATE INTER INDEX(ES).....
CREATE NONCLUSTERED INDEX [NCIX_DLI_RISK_LEVEL_THRESHOLDS] ON [VWSSTAGE].[RISK_LEVEL_THRESHOLDS] (
    [DATE_LAST_INSERTED] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STATIC TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[RISK_LEVEL_THRESHOLDS]'))
CREATE TABLE [VWSSTATIC].[RISK_LEVEL_THRESHOLDS] (
    [ID] [INT] PRIMARY KEY IDENTITY(1,1),
    [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [RISK_LEVEL_1] [VARCHAR](100) NULL,
    [RISK_LEVEL_2] [VARCHAR](100) NULL,
    [RISK_LEVEL_3] [VARCHAR](100) NULL,
    [IC_LOWER] [INT] NULL,
    [IC_UPPER] [INT] NULL,
    [HOSPITALIZATIONS_LOWER] [INT] NULL,  
    [HOSPITALIZATIONS_UPPER] [INT] NULL,  
    [VALID_FROM] [DATE] NULL
);
GO

-- 2) CREATE STATIC INDEX(ES).....
IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_RISK_LEVEL_THRESHOLDS')
CREATE NONCLUSTERED INDEX [NCIX_DLI_RISK_LEVEL_THRESHOLDS] ON [VWSSTATIC].[RISK_LEVEL_THRESHOLDS] (
    [DATE_LAST_INSERTED] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

# **STORED PROCEDURES**

---

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_RISK_LEVEL_THRESHOLDS_STAGE_TO_STATIC]
AS
BEGIN

    BEGIN TRANSACTION;

    INSERT INTO [VWSSTATIC].RISK_LEVEL_THRESHOLDS (
        [RISK_LEVEL_1],  
        [RISK_LEVEL_2],  
        [RISK_LEVEL_3],  
        [IC_LOWER],     
        [IC_UPPER],    
        [HOSPITALIZATIONS_LOWER],  
        [HOSPITALIZATIONS_UPPER],   
        [VALID_FROM]
    )
    SELECT * FROM (
        SELECT    
            -- RISK LEVELS
            LAG([DESCRIPTION],1) OVER (ORDER BY [RISK_LEVEL] ASC) AS [RISK_LEVEL_1],
            [DESCRIPTION] AS [RISK_LEVEL_2],
            LAG([DESCRIPTION],1) OVER (ORDER BY [RISK_LEVEL] DESC) AS [RISK_LEVEL_3],
            --IC THRESHOLDS
            CASE WHEN CHARINDEX('-',[7_DAYS_AVERAGE_IC_RECORDINGS]) > 0 
                THEN LEFT([7_DAYS_AVERAGE_IC_RECORDINGS], CHARINDEX('-',[7_DAYS_AVERAGE_IC_RECORDINGS])-1)
                ELSE NULL
            END AS [IC_LOWER],
            CASE WHEN CHARINDEX('-',[7_DAYS_AVERAGE_IC_RECORDINGS]) > 0 
                THEN RIGHT([7_DAYS_AVERAGE_IC_RECORDINGS], LEN([7_DAYS_AVERAGE_IC_RECORDINGS]) - CHARINDEX('-',[7_DAYS_AVERAGE_IC_RECORDINGS]))
                ELSE NULL
            END AS [IC_UPPER],
            --HOSPITAL THRESHOLDS
            CASE WHEN CHARINDEX('-',[7_DAYS_AVERAGE_HOSPITALIZATIONS]) > 0 
                THEN LEFT([7_DAYS_AVERAGE_HOSPITALIZATIONS], CHARINDEX('-',[7_DAYS_AVERAGE_HOSPITALIZATIONS])-1)
                ELSE NULL
            END AS [HOSPITALIZATIONS_LOWER],
            CASE WHEN CHARINDEX('-',[7_DAYS_AVERAGE_IC_RECORDINGS]) > 0 
                THEN RIGHT([7_DAYS_AVERAGE_HOSPITALIZATIONS], LEN([7_DAYS_AVERAGE_HOSPITALIZATIONS]) - CHARINDEX('-',[7_DAYS_AVERAGE_HOSPITALIZATIONS]))
                ELSE NULL
            END AS [HOSPITALIZATIONS_UPPER],
            -- VALIDITY
            CONVERT(datetime,[VALID_FROM],103) AS [VALID_FROM]
            FROM [VWSSTAGE].[RISK_LEVEL_THRESHOLDS]
            WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RISK_LEVEL_THRESHOLDS] )
    ) AS T0
        WHERE [IC_UPPER] IS NOT NULL AND [IC_LOWER] IS NOT NULL -- ADD ALL LOGIC TO MIDDLE LEVEL ROW

    COMMIT;
END
GO

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'RISK_LEVEL_THRESHOLDS',
        @workflow_static_name NVARCHAR(50),
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @workflow_static_name = CONCAT(N'STATIC_', @workflow_name);
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_static_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_static_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_static_name,
    @description = @workflow_description,
    @schedule = '0 12 * * *', -- At 12:00 PM everyday
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/Risico_inschalings_tabel.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/Risico_inschalings_tabel.csv'
    ELSE 'datafiles/STATIC/Risico_inschalings_tabel.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Risico niveau|Omschrijving risiconiveau|7 daags gemiddelde IC opnames|7 daags gemiddelde ziekenhuisopnames|Geldig vanaf',
    @target_columns = 'RISK_LEVEL|DESCRIPTION|7_DAYS_AVERAGE_IC_RECORDINGS|7_DAYS_AVERAGE_HOSPITALIZATIONS|VALID_FROM',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'SemiColon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
SET @source = CONCAT('dbo.SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO STATIC');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = CONCAT('PROCESS_', @workflow_name);
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_static_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_static_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO STATIC......
SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO STATIC');
SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_STATIC');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_static_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_static_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGE_TO_STATIC');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_static_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_static_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO